In [1]:
!nvidia-smi

Tue Jul  9 14:31:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%writefile sort.cu
#include <cub/cub.cuh>


void SORT(int* nums, size_t n, int* output) {
    void* d_temp_storage = nullptr;
    size_t temp_storage_bytes = 0;
    cub::DeviceRadixSort::SortKeys(d_temp_storage, temp_storage_bytes, nums, output, n);
    cudaMalloc(&d_temp_storage, temp_storage_bytes);
    cub::DeviceRadixSort::SortKeys(d_temp_storage, temp_storage_bytes, nums, output, n);
    cudaFree(d_temp_storage);
}

int main() {
    int num_elements = 1024;
    int* keys = new int[num_elements];
    int* sorted_keys = new int[num_elements];
    for (int i = 0; i < num_elements; ++i) {
        keys[i] = num_elements - i;
    }
    int* d_keys;
    int* d_sorted_keys;
    cudaMalloc(&d_keys, num_elements * sizeof(int));
    cudaMalloc(&d_sorted_keys, num_elements * sizeof(int));
    cudaMemcpy(d_keys, keys, num_elements * sizeof(int), cudaMemcpyHostToDevice);
    SORT(d_keys, num_elements, d_sorted_keys);
    cudaMemcpy(sorted_keys, d_sorted_keys, num_elements * sizeof(int), cudaMemcpyDeviceToHost);
    for (int i = 1; i < num_elements; i++) {
        std::cout << sorted_keys[i] << " ";
    }
    std::cout << std::endl;
    delete[] keys;
    delete[] sorted_keys;
    return 0;
}





Writing sort.cu


In [3]:
!nvcc -std=c++14 -arch=sm_75 sort.cu -o sort

In [4]:
!time ./sort

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 27

In [13]:
%%writefile new_sort.cu
#include <cuda_runtime.h>
#include <cub/cub.cuh>
__global__ void merge(int* d_keys, int left, int mid, int right) {
    int left_start = left;
    int right_start = mid + 1;
    int left_size = mid - left + 1;
    int right_size = right - mid;
    int* left_array = new int[left_size];
    int* right_array = new int[right_size];
    for (int i = 0; i < left_size; ++i) {
        left_array[i] = d_keys[left_start + i];
    }
    for (int j = 0; j < right_size; ++j) {
        right_array[j] = d_keys[right_start + j];
    }
    int i = 0;
    int j = 0;
    int k = left_start;
    while (i < left_size && j < right_size) {
        if (left_array[i] <= right_array[j]) {
            d_keys[k] = left_array[i];
            i++;
        } else {
            d_keys[k] = right_array[j];
            j++;
        }
        k++;
    }
    while (i < left_size) {
        d_keys[k] = left_array[i];
        i++;
        k++;
    }
    while (j < right_size) {
        d_keys[k] = right_array[j];
        j++;
        k++;
    }
    delete[] left_array;
    delete[] right_array;
}
void mergeSortHelper(int* arr, int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;
        mergeSortHelper(arr, l, m);
        mergeSortHelper(arr, m + 1, r);
        merge<<<1, 1>>>(arr, l, m, r);
    }
}
int main() {
    int num_elements = 1024;
    int* h_keys = new int[num_elements];
    int* d_keys;
    for (int i = 0; i < num_elements; ++i) {
        h_keys[i] = num_elements - i;
    }
    cudaMalloc((void**)&d_keys, num_elements * sizeof(int));
    cudaMemcpy(d_keys, h_keys, num_elements * sizeof(int), cudaMemcpyHostToDevice);
    mergeSortHelper(d_keys, 0, num_elements - 1);
    cudaMemcpy(h_keys, d_keys, num_elements * sizeof(int), cudaMemcpyDeviceToHost);
    for(int i = 0; i < num_elements; i++){
      std::cout<<h_keys[i]<<" ";
    }
    return 0;
}

Overwriting new_sort.cu


In [14]:
!nvcc -std=c++14 -arch=sm_75 new_sort.cu -o newsort

In [15]:
!time ./newsort

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 